In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

In [2]:
def generate_parity_data(n_samples=1000):
    X = np.random.randint(0, 256, size=(n_samples,))  # Random numbers from 0 to 255
    X_bin = np.array([[int(b) for b in format(x, '08b')] for x in X])  # 8-bit binary representation
    X_last_digit = X_bin[:, -1]  # Extract the last digit
    y = X % 2  # 1 if odd, 0 if even

    return torch.tensor(X_last_digit, dtype=torch.float32).unsqueeze(1), torch.tensor(y, dtype=torch.long)

# Generate training and test
X_train, y_train = generate_parity_data(800)
X_test, y_test = generate_parity_data(200)

# DataLoaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 16)  # Input size = 1 (last digit), hidden layer = 16 neurons
        self.fc2 = nn.Linear(16, 1)  # Output = 1 (binary classification)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid for binary classification
        return x


In [4]:
model = MLP()
loss_fn = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()  # Forward pass
        loss = loss_fn(y_pred, y_batch.float())  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 1/20, Loss: 0.2761
Epoch 2/20, Loss: 0.0931
Epoch 3/20, Loss: 0.0356
Epoch 4/20, Loss: 0.0187
Epoch 5/20, Loss: 0.0118
Epoch 6/20, Loss: 0.0083
Epoch 7/20, Loss: 0.0061
Epoch 8/20, Loss: 0.0048
Epoch 9/20, Loss: 0.0038
Epoch 10/20, Loss: 0.0031
Epoch 11/20, Loss: 0.0026
Epoch 12/20, Loss: 0.0022
Epoch 13/20, Loss: 0.0019
Epoch 14/20, Loss: 0.0017
Epoch 15/20, Loss: 0.0015
Epoch 16/20, Loss: 0.0013
Epoch 17/20, Loss: 0.0011
Epoch 18/20, Loss: 0.0010
Epoch 19/20, Loss: 0.0009
Epoch 20/20, Loss: 0.0008


In [17]:
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch).squeeze()
            y_pred = (y_pred > 0.5).long()  # Convert probability to 0/1
            print(X_batch[:5], y_pred[:5])
            correct += (y_pred == y_batch).sum().item()
            total += y_batch.size(0)
    return correct / total

accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [1.]]) tensor([1, 1, 1, 0, 1])
tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.]]) tensor([0, 0, 1, 0, 0])
tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.]]) tensor([0, 1, 0, 0, 0])
tensor([[0.],
        [0.],
        [1.],
        [1.],
        [1.]]) tensor([0, 0, 1, 1, 1])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [0.]]) tensor([1, 1, 1, 1, 0])
tensor([[1.],
        [1.],
        [0.],
        [0.],
        [1.]]) tensor([1, 1, 0, 0, 1])
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [1.]]) tensor([0, 0, 0, 0, 1])
Test Accuracy: 100.00%
